In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import gc
import keras

In [ ]:
import tensorflow as tf

In [ ]:
tf.test.is_gpu_available()

In [ ]:
gc.collect()

In [ ]:
%matplotlib inline

In [ ]:
train_dataset = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/train_V2.csv")
test_dataset = pd.read_csv("/kaggle/input/pubg-finish-placement-prediction/test_V2.csv")

In [ ]:
test_endset = test_dataset.copy()
pd.set_option('display.max_columns', None)

In [ ]:
train_dataset.shape

In [ ]:
train_dataset.iloc[0:4, [0, 1, 2, 3, 4, -3, -2, -1]]

In [ ]:
#对数据集进行预处理
#删除含有缺失项的样本
dataset_null = pd.isnull(train_dataset)
#print(dataset_null)
train_dataset.isnull().any()

In [ ]:
train_dataset.dropna(inplace=True)

In [ ]:
train_dataset.isnull().any()

In [ ]:
# Id group matchId 是身份索引，对于预测没有用处的所以去掉
train_dataset.drop(['Id', 'groupId', 'matchId'], axis=1, inplace=True)
train_dataset.info()

In [ ]:
train_dataset.columns #26项相关属性

In [ ]:
from time import time

In [ ]:
train_dataset['TotalDistance'] = train_dataset['rideDistance'] + train_dataset['swimDistance'] + train_dataset['walkDistance']
train_dataset.drop(['swimDistance', 'rideDistance', 'walkDistance'], axis=1, inplace=True)
#看一下处理后的数据集
train_dataset.columns

In [ ]:
#将饮料和药品统一作为一个增强属性
train_dataset['BOOSTS'] = train_dataset['boosts'] + train_dataset['heals']
train_dataset.drop(['boosts','heals'], axis=1, inplace=True)
train_dataset.columns

In [ ]:
train_dataset.head()

In [ ]:
train_dataset['matchType'].value_counts().index

In [ ]:
train_dataset['matchType'] = train_dataset['matchType'].astype('category').cat.codes
train_dataset.head()

In [ ]:
#同理，将所有的击杀都合并成一块
train_dataset['kills'] += train_dataset['headshotKills'] + train_dataset['teamKills'] + train_dataset['roadKills']
train_dataset.drop(['headshotKills','teamKills','roadKills'], axis=1, inplace=True)
train_dataset.columns

In [ ]:
train_dataset.iloc[0:5, 0:-1]

In [ ]:
train_dataset.shape

In [ ]:
test_dataset.shape

In [ ]:
test_dataset.columns

In [ ]:
# Id group matchId 是身份索引，对于预测没有用处的所以去掉
test_dataset.drop(['Id', 'groupId', 'matchId'], axis=1, inplace=True)
test_dataset.info()

In [ ]:
test_dataset['TotalDistance'] = test_dataset['rideDistance'] + test_dataset['swimDistance'] + test_dataset['walkDistance']
test_dataset.drop(['swimDistance', 'rideDistance', 'walkDistance'], axis=1, inplace=True)
#看一下处理后的数据集
test_dataset.columns

In [ ]:
#将饮料和药品统一作为一个增强属性
test_dataset['BOOSTS'] = test_dataset['boosts'] + test_dataset['heals']
test_dataset.drop(['boosts','heals'], axis=1, inplace=True)
test_dataset.columns

In [ ]:
test_dataset['matchType'] = test_dataset['matchType'].astype('category').cat.codes
test_dataset.head()

In [ ]:
#同理，将所有的击杀都合并成一块
test_dataset['kills'] += test_dataset['headshotKills'] + test_dataset['teamKills'] + test_dataset['roadKills']
test_dataset.drop(['headshotKills','teamKills','roadKills'], axis=1, inplace=True)
test_dataset.columns

In [ ]:
train_dataset.shape

In [ ]:
test_dataset.shape

In [ ]:
train_dataset.columns

In [ ]:
test_dataset.columns

In [ ]:
#对所有特征值做标准化处理

In [ ]:
train_dataset.iloc[0:5, 0:]

In [ ]:
train_dataset.head(6)

In [ ]:
test_dataset.head(5)

In [ ]:
#开始训练模型

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras import regularizers  # 正则化
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.datasets import boston_housing
from keras.layers import Dense, Dropout
from keras.utils import multi_gpu_model

In [ ]:
y = train_dataset['winPlacePerc']

In [ ]:
x = train_dataset.drop(['winPlacePerc'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=11)

In [ ]:
x_train.head(5)

In [ ]:
gc.collect()

In [ ]:
x_train_pd = pd.DataFrame(x_train)
y_train_pd = pd.DataFrame(y_train)
x_valid_pd = pd.DataFrame(x_val)
y_valid_pd = pd.DataFrame(y_val)

In [ ]:
x_valid_pd.shape

In [ ]:
y_valid_pd.shape

In [ ]:
y_valid_pd.head(5)

In [ ]:
x_train_pd.head(5)

In [ ]:
#训练集归一化
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(x_train_pd)
x_train = min_max_scaler.transform(x_train_pd)

In [ ]:
x_train

In [ ]:
x_train.shape

In [ ]:
#训练集标签归一化
min_max_scaler.fit(y_train_pd)
y_train = min_max_scaler.transform(y_train_pd)

In [ ]:
y_train

In [ ]:
#验证集归一化
min_max_scaler.fit(x_valid_pd)
x_valid = min_max_scaler.transform(x_valid_pd)

min_max_scaler.fit(y_valid_pd)
y_valid = min_max_scaler.transform(y_valid_pd)

In [ ]:
x_valid

In [ ]:
y_val

In [ ]:
y_valid

In [ ]:
#开始训练模型
model = Sequential() #初始化

In [ ]:
x_train_pd.shape[1]
#y_train_pd.shape[1]
#x_valid_pd.shape[1]
#y_valid_pd.shape[1]

In [ ]:
#添加一个输入层
model.add(Dense(units=128,
               activation='relu',
               input_shape=(x_train_pd.shape[1],)
               )
         )

In [ ]:
#丢弃神经元连接概率
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(units=256, 
                kernel_regularizer=regularizers.l2(0.01),  # 施加在权重上的正则项
                 activity_regularizer=regularizers.l1(0.01),  # 施加在输出上的正则项      
                activation='relu',
                bias_regularizer=keras.regularizers.l1_l2(0.01)  # 施加在偏置向量上的正则项
               ))

In [ ]:
model.add(Dropout(0.5))

In [ ]:
#输出层
model.add(Dense(units=1, 
               activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mae', #绝对平均误差
             optimizer='adam' #优化器
             )

In [ ]:
x_valid

In [ ]:
y_valid

In [ ]:
history = model.fit(x_train, y_train_pd, epochs=300, batch_size=20000, verbose=2,
                    validation_data = (x_valid, y_valid))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(25, 25))
#plt.figure(dpi=1080*1920)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylim(0.065,0.2)
plt.xlim(0,300)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#保存模型，加载模型

In [ ]:
from keras.utils import plot_model

In [ ]:
from keras.models import load_model

In [ ]:
model.save('MLP.h5')

In [ ]:
# 加载模型
#model = load_model('model_MLP.h5')

In [ ]:
x_test = test_dataset

In [ ]:
x_test.columns

In [ ]:
x_test_pd = pd.DataFrame(x_test)

In [ ]:
min_max_scaler.fit(x_test_pd)
x_test = min_max_scaler.transform(x_test_pd)

In [ ]:
x_test

In [ ]:
test_dataset.shape

In [ ]:
#预测 test标签
y_new = model.predict(x_test)
# y_new = model.predict(x_valid)

In [ ]:
y_new

In [ ]:
y_new.shape

In [ ]:
y_new_pd = pd.DataFrame(y_new)

In [ ]:
min_max_scaler.fit(y_new_pd)

In [ ]:
#将y_new 进行反归一化，即为预测结果
y_new = min_max_scaler.inverse_transform(y_new_pd)

In [ ]:
y_new

In [ ]:
test_endset.head(5)

In [ ]:
test_endset['winPlacePerc'] = y_new

In [ ]:
Submission = test_endset[['Id', 'winPlacePerc']]

In [ ]:
Submission.head(10)

In [ ]:
Submission.to_csv('submission.csv', index=False)